<a id="top"></a>
# Generate script

This notebook gives a documented methodology for generating scripts to automatically run MECSim over a range of parameters.

In the parameters section we set the input parameters and simulation parameters used to generate a script that will run MECSim and the python analysis codes for each parameter set. These parameters are the electrochemical parameters, e.g. $k_0$ or $E^0$, contained in the "Master.inp" text file MECSim uses to simulate a current response.

A prerequisite for running this script is to have installed [MECSim](http://www.garethkennedy.net/MECSim.html) and to have correctly set up a skeleton input file "Master.sk". The skeleton input file differs from the usual MECSim input file "Master.inp" in that it has labels for the parameters to be varied (e.g. "\$kzero" and "\$Ezero") in the correct locations. These labels must match the variable names below, **x_name**, **y_name** or more if more than 2 dimensions are required. 

This notebook will create a script that can be used to automatically run MECSim for each parameter set, compare the simulated current response to experimental data and finally run the Bayesian analysis to determine the optimal parameters as well as statistically determined error bars. More details are given in the **BayesianAnalysis.ipynb** notebook.

The contents of this notebook are:
- <p><a href="#ref_guide">Practical guide</a></p>
- <p><a href="#ref_paras">Parameters</a></p>
- <p><a href="#ref_postproc">Post-processing parameters</a></p>
- <p><a href="#ref_weights">Set weights for comparison</a></p>
- <p><a href="#ref_settings">Output settings file</a></p>
- <p><a href="#ref_prepscript">Prepare script file</a></p>
- <p><a href="#ref_genscript">Generate script and parameter loop</a></p>

<a id="ref_guide"></a>
## Practical guide


This guide will answer practical questions such as where to copy all the files? How do I run MECSim over a range of parameters?

Copy the following python scripts to the **python_dir** (default is "python"):
1. HarmonicSplitter.py
2. CompareSmoothed.py
3. BayesianAnalysis.py (optional)
4. SurfacePlotter.py (optional)

Details of how they fit together in the loop are given <a href="#ref6">here</a>.

After running this script there will be two additional files that must be put into the **script_dir** (default is "script"):
1. Master.sk
2. run_mecsim_script.sh
3. Settings.inp


![title](MECSim_ScriptingChart.PNG)


**NOTE: the skeleton input parameter file for MECSim, "Master.sk", must be setup before running the script output from this python script.**


### Load required libraries



In [1]:
import numpy as np

<a id="ref_paras"></a>
## Parameters

### Set location of the python files

Location is relative to the script that will be running. This script is required to be in the same location as the MECSim.exe file - as in the Docker image setup.

Back to <a href="#top">top</a>.

In [2]:
# python dir contains all .py files
python_dir = 'python/'
#python_dir = './'
# script dir contains script.sh (output here), Master.sk (user prepared) and Settings.inp (output here)
script_dir = 'script/'
# Master.inp dir - important for docker container vs local run
output_dir = 'output/'

### Names of variables

Name mappings for master.sk

In [3]:
x_name = '$kzero'
y_name = '$Ezero'

### Set variable ranges

In [4]:
x_min = 0.50e-2
x_max = 1.50e-2
del_x = 5.0e-3

y_min = 0.19
y_max = 0.23
del_y = 1.0e-2

<a id="ref_postproc"></a>
## Post-processing parameters

Automatic post processing. Will be added to the end of the script.

Select if the least square values in the results file will be plotted as surface plot.

Select if Bayesian analysis on the results file is to be automatically done (figures and output statistics sent to output directory)

Back to <a href="#top">top</a>.

In [5]:
# select whether surface plotter to be used
plot_surfaces = True
# select whether Bayesian statistical analysis to be used
bayesian_analysis = True

### File names

Results name - and if pre existing

In [6]:
results_name = 'results.txt'
results_exists = False
script_name = 'run_mecsim_script.sh'

### Simulation file settings

In [7]:
Simulation_output_filename = 'MECSimOutput_Pot.txt'
number_harmonics = 6  # number of harmonics (excluding the dc component - harmonic = 0)
frequency_bandwidth = 1.    # only 1 needed for simulations

### Comparison file settings

Experimental file name

In [8]:
Experimental_filename = 'MECSim_Example.txt'
needs_fft_conversion = True
Experimental_FFT_output_filename = 'ExpSmoothed.txt'
output_single_metric = True

<a id="ref_weights"></a>
## Set weights for comparison

Set weights on each harmonic (0 to $N_{harm}$) or default to equal weighting.

Metric used in "CompareSmoothed.py" is to take the smoothed harmonics from "HarmonicSplitter.py" for both the experimental and simulated data (a function of parameters run by "MECSim"), calculated the least squares difference for each harmonic and combine them via
$$
S = \sum_{j=0}^{n_{harm}} w_j LS_j
$$
where $n_{harm}$ is the number of harmonics, $j=0$ is the dc component and $LS_j$ is the relative least squares difference given by
$$
LS_j = \frac{ \sum_k^n \left( i^{exp}_k - i^{sim}_k \right)^2 }{ \sum_k^n \left( i^{exp}_k \right)^2 }
$$
where $n$ is the total number of current ($i$), time and voltage points in the smoothed experiemental ($exp$) and simiulated ($sim$) data. The weights ($w_j$) for each harmonic (and dc component) are set as a vector of any sum, or left as the unweighted default of $w_j = 1$.

Back to <a href="#top">top</a>.

### Set weights

Only needed if output_single_metric = True. Otherwise all LS$_j$ are output split by commas.

Alter the following cell for custom weights. 

In [9]:
# set to 1 if using weights (else 0)
use_weights = 1
# set weights as a numpy array with same length as number_harmonics + 1
weights = np.array([0.5,1,1,0.5,0.3,0.1,0.05])


Check that the number of weights for the number of harmonics is correct

In [10]:
if(number_harmonics+1 != len(weights)):
    print "WARNING: Found", len(weights), "weights when there are", number_harmonics+1, "harmonics (including dc)"
    if(number_harmonics+1<len(weights)):
        print "         Too many weights entered - clipping"
        weights = weights[:number_harmonics+1]
    else:
        print "         Not enough weights entered - filling with zeros"
        weights.resize(number_harmonics+1)

Convert np array to csv string (in case not python reading it in!)

In [11]:
txt_weights = ','.join(map(str, weights))
print "Weights:", txt_weights

Weights: 0.5,1.0,1.0,0.5,0.3,0.1,0.05


### Script method

In [12]:
method_type = 'grid'

<a id="ref_settings"></a>
## Output settings file

This file encodes the simulation output filename (shouldn't change), number of harmonics, bandwidth frequency and the settings for the weights.

Back to <a href="#top">top</a>.

In [13]:
f = open('Settings.inp', 'w')
f.write(Simulation_output_filename + "\t# simulation output filename\n")
f.write(str(number_harmonics) + "\t# number of harmonics\n")
f.write(str(frequency_bandwidth) + "\t# bandwidth frequency (Hz)\n")
iUseSingleMetric = 0
if(output_single_metric or output_single_metric==1):
    iUseSingleMetric = 1
f.write(str(iUseSingleMetric) + "\t# 1=use single output metric value (else=0 and each harmonic treated separately\n")
iUseWeights = 0
if(use_weights or use_weights==1):
    iUseWeights = 1
f.write(str(iUseWeights) + "\t# apply different weights to each harmonic (0 to n_harm) - if =1 then weights set below\n")
f.write(txt_weights)
f.close()

<a id="ref_prepscript"></a>
## Prepare script file

Depending on the method type selected then output a text file in bash script format for running MECSim with the analysis tools.

First set any by hand parameters. For example if you have a constant e0val=0.2 but want to keep the skeleton file general with $e0val in there. 

Note that you'll need to be careful to integrate these into the script generation yourself.

Back to <a href="#top">top</a>.

### Define function for dealing with exponential form


In [14]:

if(del_x>0):
    x_min_unit = del_x
    if(x_min>0):
        x_min_unit = min(del_x, x_min)
else:
    x_min_unit = x_min
x_exp = np.floor(np.log10(np.abs(x_min_unit))).astype(int)
print(x_exp)
x_exp_unit = 10.0**x_exp
x_min_exp = x_min / x_exp_unit
x_max_exp = x_max / x_exp_unit
del_x_exp = del_x / x_exp_unit

-3


In [15]:
def ConvertXToXExpForm(x_min, x_max, del_x):
    """
    Convert min/max/delta into a minimum common exponential form
    e.g. 1.4e-4, 1.0e-5 have 1e-5 as the minimum. 
    So the first becomes 14e-5 while the second remains as it is.
    
    Everything must be integer
    """
    if(del_x>0):
        x_min_unit = del_x
        if(x_min>0):
            x_min_unit = min(del_x, x_min)
    else:
        x_min_unit = x_min
    x_exp = np.floor(np.log10(np.abs(x_min_unit))).astype(int)
    x_exp_unit = 10.0**x_exp
    x_min_exp = int(x_min / x_exp_unit)
    x_max_exp = int(x_max / x_exp_unit)
    del_x_exp = int(del_x / x_exp_unit)
    return x_min_exp, x_max_exp, del_x_exp, x_exp

### Modify here for more than 2 dimensions

If more dimensions then add more here. Will also need to add to the parameters above and the script logic below.

In [16]:
x_min_exp, x_max_exp, del_x_exp, x_exp = ConvertXToXExpForm(x_min, x_max, del_x)
y_min_exp, y_max_exp, del_y_exp, y_exp = ConvertXToXExpForm(y_min, y_max, del_y)


In [17]:
print x_min_exp, x_max_exp, del_x_exp, x_exp
print y_min_exp, y_max_exp, del_y_exp, y_exp

5 15 5 -3
19 23 1 -2


<a id="ref_genscript"></a>
## Generate script and parameter loop

### Basic grid method

First apply HarmonicSplitter.py to split the experimental data into harmonics (dc and ac) then smooth the resultant current responses.

Next loop over all x,y combinations, each time doing:
1. Use HarmonicSplitter.py to split and smooth the harmonics
2. Use CompareSmoothed.py to compare the smoothed harmonics between the simulated current response and the experimental data.
3. Comparison is calculated as a metric (default uses least squares) which is either a composite of all harmonic data or a list of values, one for each harmonic
3. Append the x, y and metric (S) values to a single file (results_name set above)

After the loop:
1. All input parameters (x, y and any more added by user) as well as the least squared metric (S, either 1 or more values) is now stored in "results_name" (typically Results.txt)
2. BayesianAnalysis.py is run on this results file to determine the probabilities of each set of parameters given that the true fit to the experimental data exists somewhere in the chosen parameter range.
3. BayesianAnalysis will output the posterior probabilities ("posterior.txt") as well as the optimal values with error bars ("opt_parameters.txt") and plots (png and pdf) depending on the settings in BayesianAnalysis.py.
4. Additional plots of the least squares surfaces themselves are output by SurfacePlotter.py if requested.

**This script should be renamed to "run_mecsim_script.sh" and copied to "script/" along with "Settings.inp" created above.**

Back to <a href="#top">top</a>.

In [18]:
if(method_type=='grid'):
    print 'Using grid method'
    with open(script_name, "w") as text_file:
        text_file.write("#!/bin/bash\n")
        text_file.write("cp {0}Settings.inp ./\n".format(script_dir))
        # process harmonics for experimental data - if requested
        if(needs_fft_conversion):
            text_file.write("cp {0}{1} {2}\n".format(script_dir, Experimental_filename, Simulation_output_filename))
            text_file.write("python {0}HarmonicSplitter.py\n".format(python_dir))
            text_file.write("mv Smoothed.txt {0} \n".format(Experimental_FFT_output_filename))
        # setup parameter ranges
        text_file.write("xmin={0}\n".format(x_min_exp))
        text_file.write("xmax={0}\n".format(x_max_exp))
        text_file.write("xdel={0}\n".format(del_x_exp))
        text_file.write("xext=e{0}\n".format(x_exp))
        text_file.write("ymin={0}\n".format(y_min_exp))
        text_file.write("ymax={0}\n".format(y_max_exp))
        text_file.write("ydel={0}\n".format(del_y_exp))
        text_file.write("yext=e{0}\n".format(y_exp))
        text_file.write("x=$xmin\n")
        # write header for results output file - else will append
        if(not results_exists):
            text_file.write("echo '{0},{1},S' > {2}\n".format(x_name, y_name, results_name))
        # construct loop over parameters
        text_file.write("while [ $x -le $xmax ]\n")
        text_file.write("do\n")
        text_file.write("  y=$ymin\n")
        text_file.write("  while [ $y -le $ymax ]\n")
        text_file.write("  do\n")
        text_file.write("    cp {0}Master.sk ./Master.inp\n".format(script_dir))
        text_file.write("    sed -i 's/{0}/'$x$xext'/g' ./Master.inp\n".format(x_name))
        text_file.write("    sed -i 's/{0}/'$y$yext'/g' ./Master.inp\n".format(y_name))
        text_file.write("    ./MECSim.exe\n")
        text_file.write("    mv log.txt {0}log_$x$xext_$y$yext.txt\n".format(output_dir))
        text_file.write("    python {0}HarmonicSplitter.py\n".format(python_dir))
        text_file.write("    z=$(python {0}CompareSmoothed.py)\n".format(python_dir))
        text_file.write("    echo $x$xext,$y$yext,$z >> {0}\n".format(results_name))
        text_file.write("    y=$((y+ydel))\n")
        text_file.write("  done\n")
        text_file.write("  x=$((x+xdel))\n")
        text_file.write("done\n")
        # copy out raw results file
        text_file.write("cp {0} {1}\n".format(results_name, output_dir))
        # run Bayesian analysis (*.txt) and plotter (produces bayesian_plot.pdf and png) with results_name
        if(bayesian_analysis):
            text_file.write("python {0}BayesianAnalysis.py {1}\n".format(python_dir, results_name))
            # for all output file names check BayesianAnalysis.py for consistency
            text_file.write("cp bayesian_plot.* {0}\n".format(output_dir)) # plots
            text_file.write("cp posterior.txt opt_parameters.txt {0}\n".format(output_dir)) 
        # run least squares plotter (also uses results file name as argument)
        if(plot_surfaces):
            text_file.write("python {0}SurfacePlotter.py {1}\n".format(python_dir, results_name))
            text_file.write("cp surface_plot.* {0}\n".format(output_dir))


Using grid method
